In [37]:
import json
import numpy as np
import math as m

def toQt_xyzw(RPY): #Roll (X), Pitch (Y), Yaw (Z)
    if len(RPY) == 6:
        R, P, Y = [RPY[3],RPY[4],RPY[5]]
    elif len(RPY) == 3:
        R, P, Y = [RPY[0],RPY[1],RPY[2]]

    #Abbreviations for the various angular functions
    cy = m.cos(Y*0.5)
    sy = m.sin(Y*0.5)
    cp = m.cos(P*0.5)
    sp = m.sin(P*0.5)
    cr = m.cos(R*0.5)
    sr = m.sin(R*0.5)

    qw= cy*cp*cr + sy*sp*sr
    qx= cy*cp*sr - sy*sp*cr
    qy= sy*cp*sr + cy*sp*cr
    qz= sy*cp*cr - cy*sp*sr
    return ([qx, qy, qz, qw])


def toEA_RPY(qt):
    if len(qt) == 7:
        qx,qy,qz,qw = [qt[3],qt[4],qt[5],qt[6]]
    elif len(qt) == 4:
        qx,qy,qz,qw = [qt[0],qt[1],qt[2],qt[3]]
    #roll (x-axis rotation)
    sinr_cosp = 2.0*(qw*qx + qy*qz)
    cosr_cosp = 1.0 - 2.0*(qx*qx + qy*qy)
    R = m.atan2(sinr_cosp, cosr_cosp)

    #pitch (y-axis rotation)
    sinp = 2.0*(qw*qy - qz*qx)
    if (m.fabs(sinp) >= 1):
        P = m.copysign(m.pi/2, sinp) # use 90 degrees if out of range
    else:
        P = m.asin(sinp)

    #yaw (z-axis rotation)
    siny_cosp = 2.0*(qw*qz + qx*qy)
    cosy_cosp = 1.0 - 2.0*(qy*qy + qz*qz)
    Y = m.atan2(siny_cosp, cosy_cosp)
    return([R, P, Y])


def easyDEG_RPY(RPY):
    print(RPY)
    if len(RPY) == 6:
        Rdeg, Pdeg, Ydeg = [(180/m.pi)*RPY[3],(180/m.pi)*RPY[4],(180/m.pi)*RPY[5]]
    elif len(RPY) == 3:
        Rdeg, Pdeg, Ydeg = [(180/m.pi)*RPY[0],(180/m.pi)*RPY[1],(180/m.pi)*RPY[2]]
    print([Rdeg, Pdeg, Ydeg])
    if Rdeg > 180:
        Rdeg = Rdeg-360
    if Rdeg <= -180:
        Rdeg = Rdeg+360
        
    if Pdeg > 180:
        Pdeg = Pdeg-360
    if Pdeg <= -180:
        Pdeg = Pdeg+360
        
    if Ydeg > 180:
        Ydeg = Ydeg-360
    if Ydeg <= -180:
        Ydeg = Ydeg+360
    return([Rdeg, Pdeg, Ydeg])
    
def qmult(q1, q0):
    x0, y0, z0, w0 = q0
    x1, y1, z1, w1 = q1
    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)

file = "testaprilpose.json"
with open(file, 'r') as f1:
    testaprilpose_data = json.load(f1)
type(testaprilpose_data)
tag_read_loc=testaprilpose_data["TAG_0"]

tag_read = [-.5032, -.2147,1.3717,.9983,.0003,.0012,.0575]
tag_read_map_coord_XYZ = [tag_read[0],-tag_read[2],tag_read[1]]
map_coord_trans = [m.pi/2.0, 0.0, 0.0]

tag_read_map_coord_Q = qmult(tag_read[3:],toQt_xyzw(map_coord_trans))
print(tag_read_map_coord_Q)
map_coord_Q_dev = qmult(tag_read_map_coord_Q,tag_read_loc[3:])
print(easyDEG_RPY(toEA_RPY(map_coord_Q_dev)))








[-6.65246060e-01  7.46563340e-01  1.06066017e-03  6.36396103e-04]
[-3.026512365204892, -0.000736965931433605, -3.139230763476282]
[-173.4063851703968, -0.04222503751607318, -179.86467366482213]
[-173.4063851703968, -0.04222503751607318, -179.86467366482213]
